In [1]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 670.4 kB/s eta 0:02:31
   ---------------------------------------- 0.5/101.7 MB 670.4 kB/s eta 0:02:31
   ---------------------------------------- 0.8/101.7 MB 578.7 kB/s eta 0:02:55
   ---------------------------------------- 1.0/101.7 MB 662.5 kB/s eta 0:02:32
    --------------------------------------- 1.3/101.7 MB 737.4 kB/s eta 0:02:17
    --------------------------------------- 1.3/101.7 MB 737.4 kB/s eta 0:02:17
    ------------


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\asucasacav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
"""
🤖 MODELING - Credit Card Fraud Detection
Paso 1: Preparar datos y guardar en data/training/
Paso 2: Entrenar 3 modelos (LR, RF, XGB)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    confusion_matrix, roc_curve, classification_report
)
import joblib
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🚀 FASE 2: MODELING & DATA PREPARATION")
print("="*70)


🚀 FASE 2: MODELING & DATA PREPARATION


In [5]:
# ============================================================================
# FASE 1: PREPARAR DATOS
# ============================================================================
print("\n[FASE 1] Preparando datos...")

# Cargar raw data
df = pd.read_csv('../data/raw/creditcard.csv')
print(f"✅ Raw data cargado: {df.shape}")

# Copiar para no modificar original
df_processed = df.copy()

# ---- FEATURE ENGINEERING ----
print("\n   [Feature Engineering]")

# 1. Log transform de Amount (para normalizar distribución)
# Agregar 1 para evitar log(0)
df_processed['Amount_log'] = np.log1p(df_processed['Amount'])
print(f"   ✓ Log transform de Amount")

# 2. Crear features temporales de Time
# Convertir segundos a fracciones de día (0-1)
df_processed['Time_fraction'] = (df_processed['Time'] % 86400) / 86400  # Fracción del día
print(f"   ✓ Feature temporal: Time_fraction")

# ---- ESCALADO ----
print("\n   [Escalado de Features]")

# Identificar columnas a escalar (todo excepto Class)
columns_to_scale = [col for col in df_processed.columns if col != 'Class']

scaler = StandardScaler()
df_processed[columns_to_scale] = scaler.fit_transform(df_processed[columns_to_scale])

print(f"   ✓ Features escaladas con StandardScaler")
print(f"   ✓ Columnas escaladas: {len(columns_to_scale)}")

# ---- GUARDAR DATASET PROCESADO ----
print("\n   [Guardando Dataset Procesado]")

df_processed.to_csv('../data/training/features_processed.csv', index=False)
print(f"   ✓ Dataset procesado guardado: data/training/features_processed.csv")
print(f"     Shape: {df_processed.shape}")

# Guardar scaler para producción
joblib.dump(scaler, '../models/scaler.pkl')
print(f"   ✓ Scaler guardado: models/scaler.pkl")

# ---- RESUMEN DE PREPARACIÓN ----
print("\n   [Resumen de Preparación]")
print(f"   • Raw features: {df.shape[1] - 1} (excluye Class)")
print(f"   • Features engineering: +2 (Amount_log, Time_fraction)")
print(f"   • Total features: {len(columns_to_scale)}")
print(f"   • Todos los features escalados: Sí")
print(f"   • Missing values en datos procesados: {df_processed.isnull().sum().sum()}")


[FASE 1] Preparando datos...
✅ Raw data cargado: (284807, 31)

   [Feature Engineering]
   ✓ Log transform de Amount
   ✓ Feature temporal: Time_fraction

   [Escalado de Features]
   ✓ Features escaladas con StandardScaler
   ✓ Columnas escaladas: 32

   [Guardando Dataset Procesado]
   ✓ Dataset procesado guardado: data/training/features_processed.csv
     Shape: (284807, 33)
   ✓ Scaler guardado: models/scaler.pkl

   [Resumen de Preparación]
   • Raw features: 30 (excluye Class)
   • Features engineering: +2 (Amount_log, Time_fraction)
   • Total features: 32
   • Todos los features escalados: Sí
   • Missing values en datos procesados: 0


In [6]:
# ============================================================================
# FASE 2: ENTRENAR MODELOS
# ============================================================================
print("\n" + "="*70)
print("[FASE 2] Entrenando modelos...")
print("="*70)

# Usar dataset procesado para entrenar
X = df_processed.drop('Class', axis=1)
y = df_processed['Class']

print(f"\nDataset para training:")
print(f"   X shape: {X.shape}")
print(f"   y distribution: {(y==0).sum()} legítimas, {(y==1).sum()} fraudes")

# ---- SPLIT: TRAIN / TEST ----
print("\n[1] Splitting datos (70-30)...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"   Train: {X_train.shape}")
print(f"   Test: {X_test.shape}")
print(f"   Train - Legítimas: {(y_train==0).sum()}, Fraudes: {(y_train==1).sum()}")
print(f"   Test - Legítimas: {(y_test==0).sum()}, Fraudes: {(y_test==1).sum()}")

# Guardar splits procesados
X_train.to_csv('../data/training/X_train.csv', index=False)
X_test.to_csv('../data/training/X_test.csv', index=False)
y_train.to_csv('../data/training/y_train.csv', index=False)
y_test.to_csv('../data/training/y_test.csv', index=False)
print(f"   ✓ Splits guardados en data/training/")


[FASE 2] Entrenando modelos...

Dataset para training:
   X shape: (284807, 32)
   y distribution: 284315 legítimas, 492 fraudes

[1] Splitting datos (70-30)...
   Train: (199364, 32)
   Test: (85443, 32)
   Train - Legítimas: 199020, Fraudes: 344
   Test - Legítimas: 85295, Fraudes: 148
   ✓ Splits guardados en data/training/


In [7]:
# ============================================================================
# ENTRENAR MODELOS
# ============================================================================

# ---- LOGISTIC REGRESSION (BASELINE) ----
print("\n[2] Entrenando Logistic Regression (baseline)...")

lr = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    random_state=42,
    solver='lbfgs'
)

lr.fit(X_train, y_train)

y_pred_proba_lr = lr.predict_proba(X_test)[:, 1]
y_pred_lr = (y_pred_proba_lr >= 0.5).astype(int)

roc_auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
f1_lr = f1_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)

print(f"   ✅ Logistic Regression")
print(f"      ROC-AUC:  {roc_auc_lr:.4f}")
print(f"      F1-Score: {f1_lr:.4f}")
print(f"      Precision: {precision_lr:.4f}")
print(f"      Recall:   {recall_lr:.4f}")

joblib.dump(lr, '../models/lr_model.pkl')
print(f"   ✓ Modelo guardado: models/lr_model.pkl")

# ---- RANDOM FOREST ----
print("\n[3] Entrenando Random Forest...")

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=20,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

rf.fit(X_train, y_train)

y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]
y_pred_rf = (y_pred_proba_rf >= 0.5).astype(int)

roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
f1_rf = f1_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)

print(f"   ✅ Random Forest")
print(f"      ROC-AUC:  {roc_auc_rf:.4f}")
print(f"      F1-Score: {f1_rf:.4f}")
print(f"      Precision: {precision_rf:.4f}")
print(f"      Recall:   {recall_rf:.4f}")

joblib.dump(rf, '../models/rf_model.pkl')
print(f"   ✓ Modelo guardado: models/rf_model.pkl")

feature_importance_rf = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n   Top 5 features (Random Forest):")
for i, row in feature_importance_rf.head(5).iterrows():
    print(f"      {row['feature']:15s}: {row['importance']:.4f}")

# ---- XGBOOST ----
print("\n[4] Entrenando XGBoost...")

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

xgb = XGBClassifier(
    n_estimators=200,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

xgb.fit(X_train, y_train)

y_pred_proba_xgb = xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_pred_proba_xgb >= 0.5).astype(int)

roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)

print(f"   ✅ XGBoost")
print(f"      ROC-AUC:  {roc_auc_xgb:.4f}")
print(f"      F1-Score: {f1_xgb:.4f}")
print(f"      Precision: {precision_xgb:.4f}")
print(f"      Recall:   {recall_xgb:.4f}")

joblib.dump(xgb, '../models/xgb_model.pkl')
print(f"   ✓ Modelo guardado: models/xgb_model.pkl")

feature_importance_xgb = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n   Top 5 features (XGBoost):")
for i, row in feature_importance_xgb.head(5).iterrows():
    print(f"      {row['feature']:15s}: {row['importance']:.4f}")



[2] Entrenando Logistic Regression (baseline)...
   ✅ Logistic Regression
      ROC-AUC:  0.9714
      F1-Score: 0.1192
      Precision: 0.0640
      Recall:   0.8716
   ✓ Modelo guardado: models/lr_model.pkl

[3] Entrenando Random Forest...
   ✅ Random Forest
      ROC-AUC:  0.9591
      F1-Score: 0.7914
      Precision: 0.8462
      Recall:   0.7432
   ✓ Modelo guardado: models/rf_model.pkl

   Top 5 features (Random Forest):
      V14            : 0.1921
      V10            : 0.1182
      V12            : 0.1108
      V4             : 0.0963
      V17            : 0.0829

[4] Entrenando XGBoost...
   ✅ XGBoost
      ROC-AUC:  0.9745
      F1-Score: 0.8357
      Precision: 0.8864
      Recall:   0.7905
   ✓ Modelo guardado: models/xgb_model.pkl

   Top 5 features (XGBoost):
      V14            : 0.5462
      V10            : 0.0705
      V4             : 0.0629
      V12            : 0.0312
      Amount_log     : 0.0219


In [8]:
# ============================================================================
# COMPARACIÓN Y VISUALIZACIONES
# ============================================================================
print("\n" + "="*70)
print("[COMPARACIÓN] Resultados de modelos")
print("="*70)

results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'ROC-AUC': [roc_auc_lr, roc_auc_rf, roc_auc_xgb],
    'F1-Score': [f1_lr, f1_rf, f1_xgb],
    'Precision': [precision_lr, precision_rf, precision_xgb],
    'Recall': [recall_lr, recall_rf, recall_xgb]
})

print("\n" + results.to_string(index=False))

best_model_idx = results['ROC-AUC'].idxmax()
best_model_name = results.iloc[best_model_idx]['Model']
best_roc_auc = results.iloc[best_model_idx]['ROC-AUC']

print(f"\n🏆 MEJOR MODELO: {best_model_name}")
print(f"   ROC-AUC: {best_roc_auc:.4f} (Target: > 0.95)")

if best_roc_auc > 0.95:
    print(f"   ✅ TARGET ALCANZADO")
else:
    print(f"   ⚠️  Necesita ajustes")

# ---- ROC CURVES ----
print("\n[Visualizaciones] Creando gráficos...")

fig, ax = plt.subplots(figsize=(10, 6))

fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_proba_lr)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)

ax.plot(fpr_lr, tpr_lr, label=f'LR (AUC={roc_auc_lr:.4f})', linewidth=2)
ax.plot(fpr_rf, tpr_rf, label=f'RF (AUC={roc_auc_rf:.4f})', linewidth=2)
ax.plot(fpr_xgb, tpr_xgb, label=f'XGB (AUC={roc_auc_xgb:.4f})', linewidth=2)
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random')

ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title('ROC Curves - Model Comparison', fontsize=13, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/resources/images/07_roc_curves.png', dpi=300, bbox_inches='tight')
plt.close()
print("   ✅ 07_roc_curves.png")

# ---- CONFUSION MATRIX ----
if best_model_name == 'Logistic Regression':
    y_pred_best = y_pred_lr
elif best_model_name == 'Random Forest':
    y_pred_best = y_pred_rf
else:
    y_pred_best = y_pred_xgb

cm = confusion_matrix(y_test, y_pred_best)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False)
ax.set_xlabel('Predicción', fontsize=11)
ax.set_ylabel('Actual', fontsize=11)
ax.set_title(f'Confusion Matrix - {best_model_name}', fontsize=13, fontweight='bold')
ax.set_xticklabels(['Legítimo', 'Fraude'])
ax.set_yticklabels(['Legítimo', 'Fraude'])

plt.tight_layout()
plt.savefig('../reports/resources/images/08_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()
print("   ✅ 08_confusion_matrix.png")

# ---- FEATURE IMPORTANCE ----
if best_model_name == 'Random Forest':
    feature_imp = feature_importance_rf
elif best_model_name == 'XGBoost':
    feature_imp = feature_importance_xgb
else:
    feature_imp = None

if feature_imp is not None:
    top_20 = feature_imp.head(20)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(range(len(top_20)), top_20['importance'].values)
    ax.set_yticks(range(len(top_20)))
    ax.set_yticklabels(top_20['feature'].values)
    ax.set_xlabel('Importancia', fontsize=11)
    ax.set_title(f'Top 20 Features - {best_model_name}', fontsize=13, fontweight='bold')
    ax.invert_yaxis()
    
    plt.tight_layout()
    plt.savefig('../reports/resources/images/09_feature_importance_model.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("   ✅ 09_feature_importance_model.png")



[COMPARACIÓN] Resultados de modelos

              Model  ROC-AUC  F1-Score  Precision   Recall
Logistic Regression 0.971373  0.119169   0.063956 0.871622
      Random Forest 0.959133  0.791367   0.846154 0.743243
            XGBoost 0.974534  0.835714   0.886364 0.790541

🏆 MEJOR MODELO: XGBoost
   ROC-AUC: 0.9745 (Target: > 0.95)
   ✅ TARGET ALCANZADO

[Visualizaciones] Creando gráficos...
   ✅ 07_roc_curves.png
   ✅ 08_confusion_matrix.png
   ✅ 09_feature_importance_model.png


In [11]:
# ============================================================================
# GUARDAR REPORTE
# ============================================================================
print("\n[Documentación] Guardando reporte...")

reporte_md = f"""# Model Comparison Report

## Data Preparation
- **Raw data:** {df.shape}
- **After feature engineering:** {df_processed.shape}
- **New features:** Amount_log, Time_fraction
- **Scaling:** StandardScaler applied to all features

## Dataset for Training
- **Train set:** {X_train.shape[0]:,} samples
- **Test set:** {X_test.shape[0]:,} samples
- **Features:** {X.shape[1]}
- **Target balance (train):** {(y_train==0).sum():,} legítimas, {(y_train==1).sum()} fraudes
- **Target balance (test):** {(y_test==0).sum():,} legítimas, {(y_test==1).sum()} fraudes

## Model Performance

| Model | ROC-AUC | F1-Score | Precision | Recall |
|-------|---------|----------|-----------|--------|
| Logistic Regression | {roc_auc_lr:.4f} | {f1_lr:.4f} | {precision_lr:.4f} | {recall_lr:.4f} |
| Random Forest | {roc_auc_rf:.4f} | {f1_rf:.4f} | {precision_rf:.4f} | {recall_rf:.4f} |
| XGBoost | {roc_auc_xgb:.4f} | {f1_xgb:.4f} | {precision_xgb:.4f} | {recall_xgb:.4f} |

## Winner
**{best_model_name}**
- ROC-AUC: {best_roc_auc:.4f}
- Status: {"✅ TARGET ALCANZADO (> 0.95)" if best_roc_auc > 0.95 else "⚠️ Necesita ajustes"}

## Files Saved
- **Models:** `models/lr_model.pkl`, `models/rf_model.pkl`, `models/xgb_model.pkl`
- **Scaler:** `models/scaler.pkl`
- **Processed data:** `data/training/features_processed.csv`
- **Train/Test splits:** `data/training/X_train.csv`, `X_test.csv`, `y_train.csv`, `y_test.csv`
- **Visualizations:** `reports/resources/images/07_*.png`, `08_*.png`, `09_*.png`
"""

with open('../reports/model_comparison.md', 'w', encoding='utf-8') as f:
    f.write(reporte_md)

print("   ✅ reports/model_comparison.md")



[Documentación] Guardando reporte...
   ✅ reports/model_comparison.md


In [12]:
# ============================================================================
# FINAL
# ============================================================================
print("\n" + "="*70)
print("✅ MODELING COMPLETADO")
print("="*70)

print(f"""
📊 RESULTADOS:

   📁 Data Training:
      ✓ data/training/features_processed.csv
      ✓ data/training/X_train.csv
      ✓ data/training/X_test.csv
      ✓ data/training/y_train.csv
      ✓ data/training/y_test.csv

   🤖 Modelos (en models/):
      ✓ lr_model.pkl (LR)
      ✓ rf_model.pkl (RF)
      ✓ xgb_model.pkl (XGB - MEJOR)
      ✓ scaler.pkl

   📊 Gráficos (en reports/resources/images/):
      ✓ 07_roc_curves.png
      ✓ 08_confusion_matrix.png
      ✓ 09_feature_importance_model.png

   📄 Documentación:
      ✓ reports/model_comparison.md

🏆 MEJOR MODELO: {best_model_name}
   ROC-AUC: {best_roc_auc:.4f} (Target: > 0.95)

🎯 SIGUIENTE: Evaluation (03_evaluation.py)
   - Threshold optimization
   - Precision-Recall curves
   - Final metrics on test set
""")

print("="*70)





✅ MODELING COMPLETADO

📊 RESULTADOS:

   📁 Data Training:
      ✓ data/training/features_processed.csv
      ✓ data/training/X_train.csv
      ✓ data/training/X_test.csv
      ✓ data/training/y_train.csv
      ✓ data/training/y_test.csv

   🤖 Modelos (en models/):
      ✓ lr_model.pkl (LR)
      ✓ rf_model.pkl (RF)
      ✓ xgb_model.pkl (XGB - MEJOR)
      ✓ scaler.pkl

   📊 Gráficos (en reports/resources/images/):
      ✓ 07_roc_curves.png
      ✓ 08_confusion_matrix.png
      ✓ 09_feature_importance_model.png

   📄 Documentación:
      ✓ reports/model_comparison.md

🏆 MEJOR MODELO: XGBoost
   ROC-AUC: 0.9745 (Target: > 0.95)

🎯 SIGUIENTE: Evaluation (03_evaluation.py)
   - Threshold optimization
   - Precision-Recall curves
   - Final metrics on test set

